In [22]:
import pandas as pd
import numpy as np

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier

from IPython.display import Image

## Test 1
### 1) Preprocess your Titanic training data;

In [10]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
combine = [train_df, test_df]
# list(train_df)
print(train_df.columns.values)
train_df.isnull().sum()

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
# Fill NaN Mean for Age
train_df['Age'].fillna(value=train_df['Age'].mean(), inplace=True)
# Mode for Embarked
train_df['Embarked'].fillna(value=train_df['Embarked'].mode()[0], inplace=True)

# Rename Sex Value
train_df['Sex'] = train_df['Sex'].replace('female',1)
train_df['Sex'] = train_df['Sex'].replace('male',0)

# Rename Embarked Value
print(train_df['Embarked'].value_counts())    # Repeat times for S,C,Q
train_df['Embarked'] = train_df['Embarked'].replace('S',2)
train_df['Embarked'] = train_df['Embarked'].replace('C',1)
train_df['Embarked'] = train_df['Embarked'].replace('Q',0)

# train_df.isnull().sum()
train_df.head()

S    646
C    168
Q     77
Name: Embarked, dtype: int64


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,2


### 2) Select a set of important features. Please show your selected features and explain how you perform feature selection.

In [12]:
# because everyone has a different PassengerId, Name, Ticket, and there are too many NaN value on Cabine, 
# these 4 features will be not used for feature selection

def get_feature_importance():

    feature_data = train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
    label_data = train_df['Survived']
    
    feature_data['Age'].fillna(feature_data['Age'].mean(), inplace=True)
    
    # Select K best features
    k=7
    model = SelectKBest(chi2, k=k)
    X_new = model.fit_transform(feature_data, label_data)
 
    print("model shape: ",X_new.shape)
    
    # Higher Score = Highly Important Feature
    scores = model.scores_
    print('model scores:', scores)
     
    # Smaller Pvalue = Highly Important Feature
    p_values = model.pvalues_
    print('model p-values', p_values) 
 
    # From the K features, sort by Importance 
    indices = np.argsort(scores)[::-1]
    k_best_features = list(feature_data.columns.values[indices[0:k]])
    print('k best features are: ',k_best_features)
    
    return k_best_features

get_feature_importance()

model shape:  (891, 7)
model scores: [3.08736994e+01 1.70348127e+02 2.46879258e+01 2.58186538e+00
 1.00974991e+01 4.51831909e+03 2.50390659e+00]
model p-values [2.75378563e-08 6.21058490e-39 6.74051416e-07 1.08094210e-01
 1.48470676e-03 0.00000000e+00 1.13564280e-01]
k best features are:  ['Fare', 'Sex', 'Pclass', 'Age', 'Parch', 'SibSp', 'Embarked']


D:\Applications\Anaconda3\envs\pytorch_study\lib\site-packages\pandas\core\series.py:4523: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


['Fare', 'Sex', 'Pclass', 'Age', 'Parch', 'SibSp', 'Embarked']

In [13]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

### 3) Learn a decision tree model with the Titanic training data using Gini index, plot your decision tree;

In [14]:
def decision():

    train = train_df[['Fare', 'Sex', 'Pclass', 'Age', 'Parch']]
    label = train_df['Survived']

    # one-hot on features
    dic = DictVectorizer(sparse=False)
    train = dic.fit_transform(train.to_dict(orient="records"))

    print(dic.get_feature_names())
    print(train)

    # Sperate training data
    x_train, x_test, y_train, y_test = train_test_split(train, label, test_size=0.3)

    # Decision tree model
    dec = DecisionTreeClassifier(criterion="gini", max_depth=5)

    dec.fit(x_train, y_train)

    print("Accuracy：", dec.score(x_test, y_test))
    
    # export a dec tree
    export_graphviz(dec, out_file="./tree.dot", feature_names=['Fare', 'Sex', 'Pclass', 'Age', 'Parch'])

    return None

if __name__ == '__main__':
    decision()

['Age', 'Fare', 'Parch', 'Pclass', 'Sex']
[[22.          7.25        0.          3.          0.        ]
 [38.         71.2833      0.          1.          1.        ]
 [26.          7.925       0.          3.          1.        ]
 ...
 [29.69911765 23.45        2.          3.          1.        ]
 [26.         30.          0.          1.          0.        ]
 [32.          7.75        0.          3.          0.        ]]
Accuracy： 0.8022388059701493


In [15]:
# on Ubuntu terminal
# $ sudo apt install graphviz
# $ dot -Tpng  tree.dot  -o tree.png

In [20]:
Image(filename = "tree.png", width=100, height=100)

### 4) Apply the five-fold cross validation of the decision tree learning algorithm to the Titanic training data to extract average classification accuracy;

In [132]:
def decision_cv():

    train = train_df[['Fare', 'Sex', 'Pclass', 'Age', 'Parch']]
    label = train_df['Survived']

    # one-hot on features
    dic = DictVectorizer(sparse=False)
    train = dic.fit_transform(train.to_dict(orient="records"))

    print(dic.get_feature_names())
    print(train)

    # Sperate training data
    x_train, x_test, y_train, y_test = train_test_split(train, label, test_size=0.3)

    # Decision tree model
    dec = DecisionTreeClassifier(criterion="gini")
    
    param = {'max_features': [1, 2, 3, 4, 5], "max_depth":[4,5,6,7,8]}
    gc = GridSearchCV(dec, param_grid=param, cv=5)
    
    gc.fit(x_train, y_train)

    print("Accuracy:", gc.score(x_test, y_test))
    print("Score:", gc.best_score_)
    print("Estimator:",gc.best_estimator_)
    print("Result:", gc.cv_results_)
    
    

    return None

if __name__ == '__main__':
    decision_cv()

['Age', 'Fare', 'Parch', 'Pclass', 'Sex']
[[22.          7.25        0.          3.          0.        ]
 [38.         71.2833      0.          1.          1.        ]
 [26.          7.925       0.          3.          1.        ]
 ...
 [29.69911765 23.45        2.          3.          1.        ]
 [26.         30.          0.          1.          0.        ]
 [32.          7.75        0.          3.          0.        ]]
Accuracy: 0.835820895522388
Score: 0.8025935483870967
Estimator: DecisionTreeClassifier(max_depth=5, max_features=5)
Result: {'mean_fit_time': array([0.        , 0.00100002, 0.00099988, 0.00100055, 0.        ,
       0.        , 0.00099998, 0.00099988, 0.00039968, 0.00059991,
       0.00080061, 0.        , 0.00039992, 0.00100012, 0.00059996,
       0.00100002, 0.00100002, 0.00099993, 0.00039935, 0.00080066,
       0.00019937, 0.00099978, 0.00100007, 0.00060005, 0.00079989]), 'std_fit_time': array([0.00000000e+00, 1.92278339e-06, 1.51090427e-06, 2.03104644e-05,
       

### 5) Apply the five-fold cross validation of the random forest learning algorithm to the Titanic training data to extract average classification accuracy;

In [25]:
# import pandas as pd
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.tree import DecisionTreeClassifier, export_graphviz
# from sklearn.ensemble import RandomForestClassifier

def randomForest():
    # train = train_df[['Fare', 'Sex', 'Pclass', 'Age', 'Parch']]    # accuracy : 0.821
    train = train_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]    # accuracy 0.847
    label = train_df['Survived']

    # one-hot on features
    dic = DictVectorizer(sparse=False)
    train = dic.fit_transform(train.to_dict(orient="records"))

    print(dic.get_feature_names())
    print(train)

    # Sperate training data
    x_train, x_test, y_train, y_test = train_test_split(train, label, test_size=0.3)

    # rf for prediction
    rf = RandomForestClassifier()

    # n_estimators = num of trees
    param = {"n_estimators":[120,200,300,500,800,1200],
             "max_depth":[5,7,9,11,12],
             "min_samples_split":[2,3,5]}


    # Every possible parameter combination
    gc = GridSearchCV(rf, param_grid=param, cv=5)
    gc.fit(x_train,y_train)
    print("RF accuracy", gc.score(x_test,y_test))
    print("Best combination: ", gc.best_estimator_)
    

    return None


if __name__ == '__main__':
    randomForest()

['Age', 'Embarked', 'Fare', 'Parch', 'Pclass', 'Sex', 'SibSp']
[[22.          2.          7.25       ...  3.          0.
   1.        ]
 [38.          1.         71.2833     ...  1.          1.
   1.        ]
 [26.          2.          7.925      ...  3.          1.
   0.        ]
 ...
 [29.69911765  2.         23.45       ...  3.          1.
   1.        ]
 [26.          1.         30.         ...  1.          0.
   0.        ]
 [32.          0.          7.75       ...  3.          0.
   0.        ]]
RF accuracy 0.8470149253731343
Best combination:  RandomForestClassifier(max_depth=9, min_samples_split=3, n_estimators=200)


In [42]:
print(type(train_df['Survived']))
train_df['Survived'].shape

<class 'pandas.core.series.Series'>


(891,)

In [59]:
data = np.array([[1,1,1,1,1,0,0,0,1,1],[0,1,1,0,1,0,0,0,1,0],[1,1,1,0,1,0,0,0,0,0]])
df_data = pd.DataFrame(data,dtype=int).T

df_data.columns = ["A", "B", "Class Label"]

#print(x.shape) , columns = ["A", "B", "C"]
df_data

,A,B,Class Label
0,1,0,1
1,1,1,1
2,1,1,1
3,1,0,0
4,1,1,1
5,0,0,0
6,0,0,0
7,0,0,0
8,1,1,0
9,1,0,0


In [70]:
def decision():
    
    train = df_data[['A','B']]
    label = df_data['Class Label']
    # one-hot on features
    dic = DictVectorizer(sparse=False)
    train = dic.fit_transform(train.to_dict(orient="records"))

    print(dic.get_feature_names())
    print(train)

    # Sperate training data
    x_train, x_test, y_train, y_test = train_test_split(train, label, test_size=0.3)

    # Decision tree model
    dec = DecisionTreeClassifier(criterion="entropy", 
                                 max_depth=3)

    dec.fit(x_train, y_train)

    print("Accuracy：", dec.score(x_test, y_test))
    
    # export a dec tree
    export_graphviz(dec, out_file="./test.dot", feature_names=['A', 'B'])

    return None

if __name__ == '__main__':
    decision()


['A', 'B']
[[1. 0.]
 [1. 1.]
 [1. 1.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [1. 1.]
 [1. 0.]]
Accuracy： 1.0
